<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# Import python libraries 
import pystorms 
# Python Scientific Computing Stack
import numpy as np
import pandas as pd
# Plotting tools
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
# set seaborn figure preferences and colors
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.set_style("darkgrid")
colorpalette = sns.color_palette("colorblind")
colors_hex = colorpalette.as_hex()

In [3]:
plt.rcParams['figure.figsize'] = [20, 15]
plt.rcParams['figure.dpi'] = 100

# Uncontrolled Case

In [13]:
################################################################################
# Uncontrolled performance - control asset settings are set to all ones
################################################################################
env_uncontrolled = pystorms.scenarios.zeta()
actions = np.array([1.0, 1.0, 1.0, 1.0])
done = False
while not done:
    done = env_uncontrolled.step(actions)
uncontrolled_perf = sum(env_uncontrolled.data_log["performance_measure"])

In [14]:
print(uncontrolled_perf)

242393973.35370058


# Control example - BC Control

In [15]:
def bc_control(depths, actions):    
    if depths[0] >= 0:
        actions[0] = 0.2366
        actions[1] = 0.6508
        actions[2] = 0.3523
        actions[3] = 0.4303
    return actions

In [16]:
################################################################################
# Controlled performance - BC Control implementation
################################################################################
env_controllerBC = pystorms.scenarios.zeta()
done = False
currentactions = np.ones(4)
while not done:
    state = env_controllerBC.state()
    newactions = bc_control(state, currentactions)
    done = env_controllerBC.step(newactions)
    currentactions = newactions
controllerBC_perf = sum(env_controllerBC.data_log["performance_measure"])

In [17]:
print(controllerBC_perf)

284466430.56612784


# Control example - EFD Control

In [18]:
def efd_control(depths):
    TANKmax = np.argmax(depths) # index 0 is Tank1; 1 is Tank2; 2 is Tank3; 3 is Tank4; index 4 is Tank5; 5 is Tank6
    TANKmin = np.argmin(depths) # index 0 is Tank1; 1 is Tank2; 2 is Tank3; 3 is Tank4; index 4 is Tank5; 5 is Tank6
    #
    V2setting = 0.2366
    V3setting = 0.6508
    V4setting = 0.3523
    V6setting = 0.4303
    # Rule EFD01
    if (depths < 1).all():
        pass
    elif TANKmax == 1:
        V2setting = 1.0
        if TANKmin == 2:
            V3setting = 0.3159
        elif TANKmin == 3:
            V4setting = 0.1894
        else:
            V6setting = 0.1687
    elif TANKmax == 2:
        V3setting = 1.0
        if TANKmin == 1:
            V2setting = 0.1075
        elif TANKmin == 3:
            V4setting = 0.1894
        else:
            V6setting = 0.1687
    elif TANKmax == 3:
        V4setting = 1.0
        if TANKmin == 1:
            V2setting = 0.1075
        elif TANKmin == 2:
            V3setting = 0.3159
        else:
            V6setting = 0.1687
    else:
        V6setting = 1.0
        if TANKmin == 1:
            V2setting = 0.1075
        elif TANKmin == 2:
            V3setting = 0.3159
        else:
            V4setting = 0.1894
    newsettings = np.array([V2setting, V3setting, V4setting, V6setting])
    return newsettings

In [19]:
################################################################################
# Controlled performance - EFD Control implementation
################################################################################
env_controllerEFD = pystorms.scenarios.zeta()
done = False
while not done:
    state = env_controllerEFD.state()
    actions = efd_control(state)
    done = env_controllerEFD.step(actions)
controllerEFD_perf = sum(env_controllerEFD.data_log["performance_measure"])

In [20]:
print(controllerEFD_perf)

272599609.8939332


```
# Configuration file for scenario zeta

# name of scearnio
name: zeta
# state definitions
states:
      - !!python/tuple
        - "T1"
        - depthN
      - !!python/tuple
        - "T2"
        - depthN
      - !!python/tuple
        - "T3"
        - depthN
      - !!python/tuple
        - "T4"
        - depthN
      - !!python/tuple
        - "T5"
        - depthN
      - !!python/tuple
        - "T6"
        - depthN

# Action space
action_space:
      - "V2"
      - "V3"
      - "V4"
      - "V6"

# Performance Targets
performance_targets:
      # CSOs to river
      - !!python/tuple
        - "T1"
        - flooding
      - !!python/tuple
        - "T2"
        - flooding
      - !!python/tuple
        - "T3"
        - flooding
      - !!python/tuple
        - "T4"
        - flooding
      - !!python/tuple
        - "T5"
        - flooding
      - !!python/tuple
        - "CSO8"
        - flooding
      - !!python/tuple
        - "CSO10"
        - flooding
      # CSOs to creek
      - !!python/tuple
        - "T6"
        - flooding
      - !!python/tuple
        - "CSO7"
        - flooding
      - !!python/tuple
        - "CSO9"
        - flooding
      # flow to WWTP
      - !!python/tuple # Conduit that connects upstream to "Out_to_WWTP" node
        - "C14"
        - "flow"
      # control roughness
      - !!python/tuple # flow out of CSO7
        - "C5"
        - "flow"
      - !!python/tuple # flow out of CSO8
        - "C15"
        - "flow"
      - !!python/tuple # flow out of CSO9
        - "C8"
        - "flow"
      - !!python/tuple # flow out of CSO10
        - "C21"
        - "flow"
      - !!python/tuple # flow out of Tank1
        - "V1"
        - "flow"
      - !!python/tuple # flow out of Tank2
        - "V2"
        - "flow"
      - !!python/tuple # flow out of Tank3
        - "V3"
        - "flow"
      - !!python/tuple # flow out of Tank4
        - "V4"
        - "flow"
      - !!python/tuple # flow out of Tank5
        - "V5"
        - "flow"
      - !!python/tuple # flow out of Tank6
        - "V6"
        - "flow"
```